# Lezione 39 — AI nel Mondo Reale

**Obiettivi della lezione**
- Comprendere come l’AI supporta decisioni e automazione di processo.
- Chiarire il ruolo imprescindibile dell’essere umano (controllo, responsabilità, etica).
- Identificare rischi di over-delegation e errori di interpretazione.
- Collegare esempi concreti a pipeline dati e governance.

## Teoria concettuale approfondita

**Decision support**
- L’AI fornisce raccomandazioni, non decisioni finali: il responsabile umano verifica e decide.
- Output probabilistici richiedono soglie di confidenza e metriche di rischio.

**Automazione di processo**
- Esecuzione automatica di task ripetitivi (classifica ticket, estrai campi, instrada richieste).
- Richiede monitoraggio continuo: drift dei dati, metriche di qualità, fallback manuali.

**Ruolo dell’essere umano**
- Definisce il perimetro, controlla gli edge case, gestisce l’escalation.
- Oversight etico e di conformità (privacy, bias, auditabilità).

**Rischio di over-delegation**
- Accettare output senza verifica; ignorare limiti del modello; confondere correlazione con causalità.
- Automazione cieca senza controlli porta a errori amplificati in catena.

**Errori di interpretazione dell’AI**
- Scambiare un punteggio per una verità; non considerare l’incertezza; applicare modelli fuori dominio.
- Mancato allineamento tra metriche di training e metriche di business.

## Schema mentale / mappa logica
- **Quando usare AI**: volumi elevati, pattern ripetitivi, bisogno di ranking/priorità, assistenza alla decisione con verifica umana.
- **Quando NON usare**: decisioni regolamentate senza supervisione, dati critici non puliti, mancanza di metriche e logging.
- **Segnali nei dati/processi**: classi squilibrate, dati in drift, costi di errore asimmetrici → richiedono soglie e controlli.
- **Pattern operativo**: definire obiettivo, baseline manuale, addestrare/validare, monitorare drift, definire fallback manuale, audit periodico.

## Notebook dimostrativo (simulazione controllo soglia e fallback umano)
Useremo un modello fittizio di classificazione ticket con punteggi di confidenza. Mostriamo:
- decisione automatica solo se la confidenza supera una soglia;
- instradamento a operatore umano in caso contrario;
- logging per audit.

In [ ]:
# Simuliamo un set di ticket con punteggi di confidenza generati da un modello fittizio
import pandas as pd

pd.set_option("display.precision", 3)

data = pd.DataFrame(
    {
        "ticket_id": [1, 2, 3, 4, 5],
        "testo": [
            "Richiesta rimborso per spedizione in ritardo",
            "Errore fatturazione importo duplicato",
            "Info su tempi di consegna standard",
            "Reclamo su prodotto danneggiato",
            "Domanda generica su resi"
        ],
        "score_modello": [0.92, 0.41, 0.63, 0.78, 0.35],
    }
)

# Etichetta target fittizia: 1 = escalation a finance, 0 = altri reparti
# Qui non addestriamo: ci focalizziamo su decisioni soglia/fallback
soglia_auto = 0.8  # confidenza richiesta per automatizzare


In [ ]:
# Funzione che decide automazione vs escalation umana in base alla soglia

def instrada_ticket(row, soglia):
    if row["score_modello"] >= soglia:
        return "auto_finance"  # automatizzato
    return "review_umano"      # passa a operatore


data["azione"] = data.apply(instrada_ticket, axis=1, soglia=soglia_auto)
data

In [ ]:
# Calcoliamo una metrica semplice di copertura automatica
copertura_auto = (data["azione"] == "auto_finance").mean()
copertura_auto

In [ ]:
# Possiamo simulare l'impatto della soglia sulla copertura automatica

for soglia in [0.5, 0.7, 0.9]:
    azioni = data.apply(instrada_ticket, axis=1, soglia=soglia)
    cop = (azioni == "auto_finance").mean()
    print(f"soglia={soglia:.1f} -> copertura auto={cop:.2f}")

### Osservazioni sul risultato
- Soglia alta → meno automazione, più controllo umano; soglia bassa → più automazione ma maggior rischio di errore.
- Necessario bilanciare costi di errore (falsi positivi vs falsi negativi) con capacità operativa del team.
- Logging delle azioni permette audit e tuning continuo.

## Esercizi svolti (step-by-step)
Esercizi per consolidare: definizione di soglie, logging, gestione drift e escalation.

In [ ]:
# Esercizio 1: aggiungere un controllo di drift semplice sullo score medio

# ipotizziamo uno stream settimanale di score medi
score_medio_storico = [0.78, 0.81, 0.79, 0.8]
score_medio_corrente = 0.68  # improvviso calo

soglia_drift = 0.1  # variazione assoluta tollerata

def drift_alert(storic, corrente, delta):
    baseline = sum(storic) / len(storic)
    return abs(corrente - baseline) > delta

alert = drift_alert(score_medio_storico, score_medio_corrente, soglia_drift)
alert

In [ ]:
# Esercizio 2: registrare audit log per ogni decisione (auto vs umano)

log = []
for _, row in data.iterrows():
    log.append(
        {
            "ticket_id": row["ticket_id"],
            "score": row["score_modello"],
            "azione": row["azione"],
            "note": "" if row["azione"] == "auto_finance" else "richiede verifica"
        }
    )
log_df = pd.DataFrame(log)
log_df

In [ ]:
# Esercizio 3: simulare una policy di escalation su classi ad alto costo di errore

alto_costo = ["rimborso", "fatturazione", "danneggiato"]

def necessita_escalation(testo):
    return any(term in testo.lower() for term in alto_costo)

policy = []
for _, row in data.iterrows():
    if necessita_escalation(row["testo"]):
        policy.append("forza_review")
    else:
        policy.append(row["azione"])

data["policy_finale"] = policy
data[["ticket_id", "testo", "score_modello", "azione", "policy_finale"]]

## Conclusione operativa
- Portarsi a casa: l’AI in produzione richiede soglie, logging, monitoraggio, policy di escalation e ruolo umano chiaro.
- Errori da evitare: automazione cieca, metriche scollegate dal business, assenza di audit e gestione drift.
- Ponte verso la prossima lezione: Recap finale e mappa completa per scegliere strumenti e approcci.